In [2]:
import copy
import japanize_matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy 
from matplotlib.pyplot import imshow
from numpy.random import randn
from scipy import stats

In [3]:
def liner(X, y):
    """linear regression

    Args:
        X (_type_): 説明変数
        y (_type_): target 
    """
    p = X.shape[1]
    x_bar = [np.mean(X[:, j]) for j in range(p)]
    X = [X[:, j]-x_bar[j] for j in range(p)]
    
    y_bar = np.mean(y)
    y = y - y_bar
    
    beta = np.dot(np.linealg.inv(np.dot(X.T, X)), np.dot(X.T, y))
    
    beta_0 = y_bar - np.dot(x_bar, beta)
    
    return beta, beta_0

In [5]:
def soft_th(lam, x):
    return np.sign(x)*np.maximum(np.abs(x)-lam, np.zeros(1))

def centralize(X0, y0, standardize=True):
    X = copy.copy(X0)
    y = copy.copy(y0)
    n, p = X.shape
    X_bar = np.zeros(p)
    X_sd  = np.zeros(p)

    for j in range(p):
        X_bar[j] = np.mean(X[:, j])
        X[:, j]  = X[:, j] - X_bar[j]
        X_sd[j]  = np.std(X[:, j])
        if standardize is True:
            X[:, j] = X[:, :] - X_bar[j]
            
        if np.ndim(y) == 2:
            K = y.shape[1]
            y_bar = np.zeros(K)
            for k in range(K):
                y_bar[k] = np.mean(y[:,k])
                y[:, k] = y[:, k] - y_bar[k]
                
        else:
            y_bar = np.mean(y)
            y = y - y_bar
        return X, y, X_bar, X_sd, y_bar
    
def linear_lasso(X, y, lam=0, beta=None):
    n, p = X.shape
    
    if beta is None:
        beta = np.zeros(p)
        
    X, y, X_bar, X_sd, y_bar = centralize(X, y)
    eps = 1
    beta_old = copy.copy(beta)
    
    while eps > 0.00001:
        for j in range(p):
            r = y
            for k in range(p):
                if j != k:
                    r = r - X[:, k]*beta[k]
            z = (np.dot(r, X[:, k])/n)
            beta[j] = soft_th(lam, z)
        eps = np.linalg.norm(beta - beta_old, 2)
        beta_old = copy.copy(beta)
    beta = beta/X_sd
    beta_0 = y_bar - np.dot(X_bar, beta)
    
    return beta, beta_0
                    